In [11]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib, pymysql, calendar, time, json
import pymysql
from urllib.request import urlopen
from datetime import datetime
from threading import Timer
import pymysql

conn = pymysql.connect(host='localhost', port=3307, db='investar', user='root', passwd='asdf1038', charset='utf8')

with conn.cursor() as curs:
    sql = """
    create table if not exists company_info(
        code varchar(20),
        company varchar(40),
        last_update date,
        primary key (code))
    """
    curs.execute(sql)
    sql = """
    create table if not exists daily_price(
        code varchar(20),
        date DATE,
        open bigint(20),
        high bigint(20),
        low bigint(20),
        close bigint(20),
        diff bigint(20),
        volume bigint(20),
        primary key (code, date))
    """
    curs.execute(sql)
conn.commit()
codes = dict()

In [14]:
def read_krx_code():
    """KRX로부터 상장법인목록 파일을 읽어와서 데이터프레임으로 반환"""
    url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&serachType=13'
    krx = pd.read_html(url, header=0)[0]
    krx = krx[['종목코드', '회사명']]
    krx = krx.rename(columns={'종목코드':'code', '회사명':'company'})
    krx.code = krx.code.map('{:06d}'.format)
    return krx

temp = read_krx_code()
temp

,code,company
0,005560,JS전선
1,101380,거북선2호
2,114140,거북선6호
3,064900,교보메리츠
4,028780,국제관광공사
...,...,...
3883,199800,툴젠
3884,222670,플럼라인생명과학
3885,239890,피엔에이치테크
3886,331660,한국미라클피플사


In [17]:
import re
start_date = '2020 year 8/30'
start_lst = re.split('\D+', start_date)
start_year = int(start_lst[0])
start_month = int(start_lst[1])
start_day = int(start_lst[2])
start_date = f"{start_year:04d}-{start_month:02d}-{start_day:02d}"
print('start_date: ', start_date)

start_date:  2020-08-30
